In [2]:
# ===========================
# ENCODER + PLOTTING: TITIK SAMPLING & NOMOR (untuk data ECG sintetis)
# - Disesuaikan untuk file .npz:
#     signal : (T,)
#     t      : (T,)
#     fs     : scalar (opsional, akan dihitung jika tidak ada)
#     r_time : scalar (opsional, jika tidak ada → pakai argmax(|signal|))
#
# Struktur folder:
#   ROOT_DIR/
#       N/beat_0000.npz
#       S/...
#       V/...
#       F/...
#       Q/...
#
# Hasil encoding:
#   OUT_DIR/<kelas>/beat_0000_sampler_centered_var.npz
#
# Hasil plotting titik+nomor:
#   FIGS_DIR/<kelas>/beat_0000_raw_points_numbers_only.{png,pdf,svg}
# ===========================
import os
from glob import glob
from typing import List, Tuple, Optional, Dict, Sequence, Iterable
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

# ---------- Konfigurasi ----------
ROOT_DIR  = "dataset_ecg"                  # folder data sintetis per kelas
OUT_DIR   = "spikes_sampler_centered_var"  # output sampler NPZ (N_anchor × SLOTS)
FIGS_DIR  = "figs_sampler_var"            # output gambar contoh

# Jumlah titik per region (total N_anchor = N_LEFT + N_QRS + N_RIGHT)
N_LEFT  = 15
N_QRS   = 9
N_RIGHT = 6

# Jendela waktu relatif ke R=0 (detik)
LEFT_WIN  = (-0.20, -0.05)
RIGHT_WIN = ( 0.06,  0.35)

# QRS (pakai OFFSETS eksak ATAU auto simetris dgn SPAN)
QRS_OFFSETS_S = [-0.032, -0.024, -0.016, -0.008, 0.000, 0.008, 0.016, 0.024, 0.032]
QRS_SPAN_S    = 0.015

# Bias kepadatan titik P/T (gamma>1 → makin rapat ke arah R)
LEFT_GAMMA  = 1.0
RIGHT_GAMMA = 1.0

# ----- Spike slots & “rate” kontrol -----
SLOTS                 = 64
SIGN_MODE             = "abs" # "abs" | "rectify_pos" | "bipolar"
RATE_GAMMA            = 1.0
RATE_SCALE            = 1.0
MIN_SPIKES_PER_ANCHOR = 0
MAX_SPIKES_PER_ANCHOR = 32

# Demo
SEED = 7

# ===== Kontrol penomoran =====
PER_LABEL_DY_PT: Dict[int, float] = {}
PER_LABEL_DY_LIST: Optional[Sequence[Optional[float]]] = None
STRICT_DY = False

# Nomor label yang TIDAK ditampilkan (1-based)
EXCLUDED_LABELS: Sequence[int] = (2, 4, 6, 8, 10, 12, 14, 16)

# Posisi khusus per label (1-based)
ORIENT_OVERRIDES: Dict[int, str] = {
    17: "below",
    18: "left",
    19: "left",
    20: "above",
    21: "right",
    22: "left",
    23: "below",
}
# Offset KECIL supaya dekat titik
ORIENT_OFFSETS_PT: Dict[str, Tuple[int, int, str, str]] = {
    # (dx_pt, dy_pt, ha, va)
    "above": (0, 5, "center", "bottom"),
    "below": (0,-5, "center", "top"),
    "left":  (-5, 0, "right",  "center"),
    "right": (5, 0, "left",    "center"),
}

# Padding sumbu-Y tambahan agar label tidak terpotong
YPAD_TOP_FRAC    = 0.12  # headroom atas
YPAD_BOTTOM_FRAC = 0.06

# =========================
# IEEE plotting helpers
# =========================
def set_ieee_style(dpi=600):
    import matplotlib as mpl
    base = {
        "figure.dpi": dpi,
        "savefig.dpi": dpi,
        "font.family": "serif",
        "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
        "mathtext.fontset": "dejavuserif",
        "axes.titlesize": 8,
        "axes.labelsize": 8,
        "xtick.labelsize": 8,
        "ytick.labelsize": 8,
        "legend.fontsize": 8,
        "axes.linewidth": 0.8,
        "xtick.major.width": 0.6,
        "ytick.major.width": 0.6,
        "xtick.minor.width": 0.5,
        "ytick.minor.width": 0.5,
        "grid.linewidth": 0.4,
        "svg.fonttype": "none",
        "pdf.fonttype": 42,
    }
    for k, v in base.items():
        if k in mpl.rcParams:
            mpl.rcParams[k] = v

# ---------- Helpers ----------
def _safe_label_from_path(p: str) -> str:
    return os.path.basename(os.path.dirname(p))

def _load_raw_numeric(npz_path: str):
    """
    Loader SERBAGUNA:
    - Jika file punya 'x' & 't_rel' (format lama MIT-BIH) → gunakan langsung.
    - Jika file punya 'signal' & 't' (format data sintetis) → konversi ke:
        x[0]  = signal
        t_rel = t - r_time, dengan:
          * kalau ada 'r_time' → pakai itu
          * kalau tidak ada   → deteksi dari argmax(|signal|)
    """
    d = np.load(npz_path, allow_pickle=True)

    # ---- Format lama (MIT-BIH) ----
    if "x" in d.files and "t_rel" in d.files:
        x = d["x"]         # (2,T) MLII,V1
        t_rel = d["t_rel"] # (T,)
        fs = int(d["fs"])
        d.close()
        return x, t_rel.astype(np.float32), fs

    # ---- Format baru: ECG sintetis ----
    if "signal" not in d.files or "t" not in d.files:
        d.close()
        raise ValueError(f"File {npz_path} tidak berisi 'x,t_rel' maupun 'signal,t'.")

    sig = d["signal"].astype(np.float32)   # (T,)
    t   = d["t"].astype(np.float32)        # (T,)

    if "fs" in d.files:
        fs = float(d["fs"])
    else:
        # Hitung dari t (asumsi uniform)
        if len(t) < 2:
            raise ValueError(f"t terlalu pendek untuk infer fs di {npz_path}")
        dt = float(t[1] - t[0])
        fs = 1.0 / dt

    if "r_time" in d.files:
        r_time = float(d["r_time"])
    else:
        # fallback: deteksi R-peak dari maksimum amplitudo
        r_idx = int(np.argmax(np.abs(sig)))
        r_time = float(t[r_idx])

    d.close()

    # R-peak di t_rel=0
    t_rel = t - r_time

    # x[0] = MLII; untuk kompatibilitas dengan kode lama, buat x shape (2,T)
    x0 = sig
    x1 = sig  # dummy, tidak dipakai
    x  = np.stack([x0, x1], axis=0)  # (2,T)

    return x, t_rel.astype(np.float32), int(round(fs))

def list_all_npz(root: str) -> List[str]:
    paths: List[str] = []
    for c in ["N","S","V","F","Q"]:
        paths += glob(os.path.join(root, c, "*.npz"))
    return sorted(paths)

# ---------- Anchor tools ----------
def _center_index_from_t(t_rel: np.ndarray) -> int:
    return int(np.argmin(np.abs(t_rel)))

def _nonlinear_linspace(a: float, b: float, n: int, gamma: float = 1.0, bias: str = "end") -> np.ndarray:
    if n <= 0:
        return np.array([], dtype=np.float32)
    if n == 1:
        return np.array([0.5*(a+b)], dtype=np.float32)
    u = np.linspace(0.0, 1.0, n)
    if gamma <= 0: gamma = 1.0
    if bias == "start":
        u = 1.0 - (1.0 - u)**gamma
    else:
        u = u**gamma
    return (a + (b - a) * u).astype(np.float32)

def _anchor_indices_centered_variable(
    t_rel: np.ndarray, fs: int,
    left_win=(-0.30, -0.05), right_win=(0.05, 0.45),
    n_left: int = 5, n_qrs: int = 5, n_right: int = 5,
    qrs_offsets_s: Optional[List[float]] = None,
    qrs_span_s: float = 0.020,
    left_gamma: float = 1.0, right_gamma: float = 1.0
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    iR = _center_index_from_t(t_rel)
    tR = float(t_rel[iR])  # ≈0

    aL, bL = tR + left_win[0],  tR + left_win[1]
    aR, bR = tR + right_win[0], tR + right_win[1]

    left_ts  = _nonlinear_linspace(aL, bL, n_left,  gamma=left_gamma,  bias="end")
    if qrs_offsets_s is not None and len(qrs_offsets_s) > 0:
        qrs_ts = np.array([tR + float(d) for d in qrs_offsets_s], dtype=np.float32)
    else:
        if n_qrs <= 0:
            qrs_ts = np.array([], dtype=np.float32)
        elif n_qrs == 1:
            qrs_ts = np.array([tR + 0.0], dtype=np.float32)
        else:
            qrs_ts = np.linspace(tR - qrs_span_s, tR + qrs_span_s, n_qrs).astype(np.float32)
    right_ts = _nonlinear_linspace(aR, bR, n_right, gamma=right_gamma, bias="start")

    t_anchor = np.concatenate([left_ts, qrs_ts, right_ts]).astype(np.float32)
    regions  = np.concatenate([
        np.zeros(len(left_ts), dtype=np.int8),
        np.ones(len(qrs_ts),  dtype=np.int8),
        np.full(len(right_ts), 2, dtype=np.int8)
    ])
    idx_anchor = np.array([int(np.argmin(np.abs(t_rel - ta))) for ta in t_anchor], dtype=np.int32)
    return idx_anchor, t_anchor, regions

# ---------- Amplitudo → slots spikes ----------
def _amplitude_to_slots_spikes(
    amp: float,
    amp_max: float,
    slots: int,
    min_spikes: int = 0,
    max_spikes: Optional[int] = None,
    sign_mode: str = "abs",
    rate_gamma: float = 1.0,
    rate_scale: float = 1.0
) -> np.ndarray:
    if slots <= 0:
        return np.zeros(0, dtype=np.uint8)
    if amp_max <= 1e-12:
        r = 0.0
    else:
        if sign_mode == "rectify_pos":
            r = float(np.clip(max(amp, 0.0) / amp_max, 0.0, 1.0))
        else:
            r = float(np.clip(abs(amp) / amp_max, 0.0, 1.0))
    rg = rate_gamma if rate_gamma and rate_gamma > 0 else 1.0
    r = (r ** rg) * float(rate_scale)
    r = float(np.clip(r, 0.0, 1.0))
    if max_spikes is None:
        max_spikes = slots
    k = int(np.round(r * slots))
    k = max(min_spikes, min(k, max_spikes))
    v = np.zeros(slots, dtype=np.uint8)
    if k == 0:
        return v
    pos = np.linspace(0, slots - 1, num=k, endpoint=True)
    idx = np.unique(np.rint(pos).astype(int))
    if sign_mode == "bipolar":
        if amp >= 0:
            idx = np.clip(idx - (idx % 2), 0, slots - 1)            # genap
        else:
            idx = np.clip(idx - ((idx + 1) % 2) + 1, 0, slots - 1)  # ganjil
    v[idx] = 1
    return v

# ---------- Build sampler ----------
def build_sampler_var_ml2(
    x: np.ndarray, t_rel: np.ndarray, fs: int,
    left_win=(-0.30, -0.05), right_win=(0.05, 0.45),
    n_left: int = 5, n_qrs: int = 5, n_right: int = 5,
    qrs_offsets_s: Optional[List[float]] = None,
    qrs_span_s: float = 0.020,
    left_gamma: float = 1.0, right_gamma: float = 1.0,
    slots: int = 128,
    min_spikes_per_anchor: int = 0,
    max_spikes_per_anchor: Optional[int] = None,
    sign_mode: str = "abs",
    rate_gamma: float = 1.0,
    rate_scale: float = 1.0
):
    ml2 = x[0]  # kita pakai lead pertama (sinyal sintetis)
    idx_anchor, t_anchor, regions = _anchor_indices_centered_variable(
        t_rel, fs,
        left_win=left_win, right_win=right_win,
        n_left=n_left, n_qrs=n_qrs, n_right=n_right,
        qrs_offsets_s=qrs_offsets_s, qrs_span_s=qrs_span_s,
        left_gamma=left_gamma, right_gamma=right_gamma
    )
    N = len(idx_anchor)
    amax = float(np.max(np.abs(ml2))) + 1e-12
    S = np.zeros((N, slots), dtype=np.uint8)
    for j, idx in enumerate(idx_anchor):
        amp = float(ml2[int(idx)])
        S[j] = _amplitude_to_slots_spikes(
            amp=amp, amp_max=amax, slots=slots,
            min_spikes=min_spikes_per_anchor,
            max_spikes=max_spikes_per_anchor,
            sign_mode=sign_mode,
            rate_gamma=rate_gamma,
            rate_scale=rate_scale
        )
    names = [f"MLII_anchor_{k+1:02d}" for k in range(N)]
    meta = {"anchor_indices": idx_anchor, "anchor_times": t_anchor, "anchor_regions": regions, "slots": int(slots)}
    return S, names, meta

def save_spike_grid_npz(out_path: str, t_rel: np.ndarray, fs: int, label: str,
                        spikes: np.ndarray, ch_names: List[str], **extra):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    np.savez(out_path,
        t_rel=t_rel.astype(np.float32),
        fs=np.int32(fs),
        label=np.array(str(label), dtype=np.unicode_),
        spikes=spikes.astype(np.uint8),
        ch_names=np.array(ch_names, dtype=np.unicode_),
        **{k: v for k, v in extra.items()}
    )

# ---------- Penomoran (dekat titik, orientasi khusus) ----------
def _annotate_anchors_neat_v6(
    ax, t, y, idx_anchor, *,
    fontsize=6.5,
    dy_steps_pt=(3, -3, 4, -4, 5, -5),
    marker_safe_r_px=5.0,
    per_label_dy_pt: Optional[Dict[int, float]] = None,
    per_label_dy_list: Optional[Sequence[Optional[float]]] = None,
    strict: bool = False,
    excluded_labels: Iterable[int] = (),
    orient_overrides: Optional[Dict[int, str]] = None,
    orient_offsets: Optional[Dict[str, Tuple[int,int,str,str]]] = None
):
    fig = ax.figure
    fig.canvas.draw()
    renderer = fig.canvas.get_renderer()
    placed_bboxes = []
    ax_bbox_disp = ax.get_window_extent(renderer=renderer)
    excluded = set(int(n) for n in excluded_labels)
    orient_overrides = orient_overrides or {}
    orient_offsets = orient_offsets or ORIENT_OFFSETS_PT

    def _overlap(r1, r2):
        (x10,y10,x11,y11) = r1
        (x20,y20,x21,y21) = r2
        return not (x11 < x20 or x21 < x10 or y11 < y20 or y21 < y10)

    def _get_custom_dy(j1_based: int) -> Optional[float]:
        if per_label_dy_list is not None:
            idx = j1_based - 1
            if 0 <= idx < len(per_label_dy_list):
                return per_label_dy_list[idx]
        if per_label_dy_pt is not None and j1_based in per_label_dy_pt:
            return per_label_dy_pt[j1_based]
        return None

    order = np.argsort(idx_anchor)  # urut waktu
    for j in order:
        label_num = int(j + 1)  # 1-based
        i = int(idx_anchor[j])
        x0, y0 = float(t[i]), float(y[i])

        # marker selalu
        ax.plot(x0, y0, marker="o", markersize=3.0, color="0.1",
                markeredgecolor="1.0", markeredgewidth=0.3, zorder=7)

        if label_num in excluded:
            continue

        xpix, ypix = ax.transData.transform((x0, y0))

        # ==== 1) ORIENTASI KHUSUS ====
        if label_num in orient_overrides:
            ori = orient_overrides[label_num].lower().strip()
            dx_pt, dy_pt, ha, va = orient_offsets.get(ori, (0, 5, "center", "bottom"))
            marker_r = 3.0
            tries = [(dx_pt, dy_pt)]
            tweaks = [(-2,0),(2,0),(0,-2),(0,2),(-3,0),(3,0),(0,-3),(0,3)]
            tries += [(dx_pt+tx, dy_pt+ty) for tx,ty in tweaks]
            placed = False
            for dx_try, dy_try in tries:
                ann = ax.annotate(
                    str(label_num), xy=(x0, y0), xytext=(dx_try, dy_try),
                    textcoords="offset points", fontsize=fontsize,
                    ha=ha, va=va, zorder=6, annotation_clip=True,
                    bbox=None, path_effects=[pe.withStroke(linewidth=0.8, foreground="white")],
                    arrowprops=None
                )
                fig.canvas.draw()
                bb = ann.get_window_extent(renderer=renderer).expanded(1.02, 1.06)
                mx0, my0 = xpix - marker_r, ypix - marker_r
                mx1, my1 = xpix + marker_r, ypix + marker_r
                overlap_marker = _overlap((bb.x0, bb.y0, bb.x1, bb.y1), (mx0,my0,mx1,my1))
                overlap_label  = any(_overlap((bb.x0,bb.y0,bb.x1,bb.y1),
                                              (b.x0,b.y0,b.x1,b.y1)) for b in placed_bboxes)
                out_bot   = bb.y0 < ax_bbox_disp.y0
                out_top   = bb.y1 > ax_bbox_disp.y1
                if not overlap_marker and not overlap_label and not out_bot and not out_top:
                    placed_bboxes.append(bb); placed = True; break
                ann.remove()
            if placed:
                continue

        # ==== 2) DEFAULT MINI ====
        ha, va = "center", "bottom"
        custom_dy = _get_custom_dy(label_num)
        if strict and custom_dy is not None:
            dy_candidates = [int(custom_dy)]
        else:
            dy_candidates = list(dy_steps_pt)
            if custom_dy is not None:
                dy_candidates = [int(custom_dy)] + dy_candidates

        chosen = None
        for dy_pt in dy_candidates:
            ann = ax.annotate(
                str(label_num), xy=(x0, y0), xytext=(0, dy_pt),
                textcoords="offset points", fontsize=fontsize,
                ha=ha, va=va, zorder=6, annotation_clip=True,
                bbox=None, path_effects=[pe.withStroke(linewidth=0.8, foreground="white")],
                arrowprops=None
            )
            fig.canvas.draw()
            bb = ann.get_window_extent(renderer=renderer).expanded(1.02, 1.06)
            mx0, my0 = xpix - marker_safe_r_px, ypix - marker_safe_r_px
            mx1, my1 = xpix + marker_safe_r_px, ypix + marker_safe_r_px
            overlap_marker = _overlap((bb.x0, bb.y0, bb.x1, bb.y1), (mx0,my0,mx1,my1))
            overlap_label  = any(_overlap((bb.x0,bb.y0,bb.x1,bb.y1),
                                          (b.x0,b.y0,b.x1,b.y1)) for b in placed_bboxes)
            out_bot   = bb.y0 < ax_bbox_disp.y0
            out_top   = bb.y1 > ax_bbox_disp.y1
            if not overlap_marker and not overlap_label and not out_bot and not out_top:
                chosen = (ann, bb); break
            ann.remove()

        if chosen is None:
            ann = ax.annotate(
                str(label_num), xy=(x0, y0), xytext=(0, 4),
                textcoords="offset points", fontsize=fontsize,
                ha="center", va="bottom", zorder=6, annotation_clip=True,
                bbox=None, path_effects=[pe.withStroke(linewidth=0.8, foreground="white")],
                arrowprops=None
            )
            fig.canvas.draw()
            bb = ann.get_window_extent(renderer=renderer).expanded(1.02, 1.06)

        placed_bboxes.append(bb)

# ---------- Plot ECG + HANYA TITIK & NOMOR ----------
def plot_raw_points_and_numbers_only(
    raw_npz_path: str,
    sampler_npz_path: Optional[str] = None,
    save_dir: str = "figs_sampler_centered_var",
    show: bool = False,
    left_win=(-0.30, -0.05), right_win=(0.05, 0.45),
    n_left: int = 5, n_qrs: int = 5, n_right: int = 5,
    qrs_offsets_s: Optional[List[float]] = None,
    qrs_span_s: float = 0.020,
    left_gamma: float = 1.0, right_gamma: float = 1.0,
    per_label_dy_pt: Optional[Dict[int,float]] = None,
    per_label_dy_list: Optional[Sequence[Optional[float]]] = None,
    strict: bool = False,
    excluded_labels: Sequence[int] = EXCLUDED_LABELS,
    orient_overrides: Dict[int, str] = ORIENT_OVERRIDES,
    orient_offsets: Dict[str, Tuple[int,int,str,str]] = ORIENT_OFFSETS_PT
):
    set_ieee_style(dpi=600)

    x, t_rel, fs = _load_raw_numeric(raw_npz_path)
    label = _safe_label_from_path(raw_npz_path)
    base = os.path.splitext(os.path.basename(raw_npz_path))[0]
    ml2 = x[0]

    # t_abs supaya sumbu-x “time (s)” kelihatan natural
    # t_rel sudah center di R=0, jadi t_abs = t_rel + r0 (kita set r0=0)
    t = t_rel

    # Hitung indeks anchor
    if sampler_npz_path is not None and os.path.exists(sampler_npz_path):
        d = np.load(sampler_npz_path, allow_pickle=True)
        idx_anchor = d["anchor_indices"].astype(int) if "anchor_indices" in d.files else None
        d.close()
    else:
        idx_anchor = None
    if idx_anchor is None:
        idx_anchor, _, _ = _anchor_indices_centered_variable(
            t_rel, fs,
            left_win=left_win, right_win=right_win,
            n_left=n_left, n_qrs=n_qrs, n_right=n_right,
            qrs_offsets_s=qrs_offsets_s, qrs_span_s=qrs_span_s,
            left_gamma=left_gamma, right_gamma=right_gamma
        )

    fig = plt.figure(figsize=(6.8, 3.0))
    ax = fig.add_subplot(1,1,1)

    ax.plot(t, ml2, lw=0.9, color="0.1")
    ax.set_xmargin(0.02)
    ax.margins(y=0.05)

    ymin, ymax = ax.get_ylim()
    yr = ymax - ymin if ymax > ymin else 1.0
    ax.set_ylim(ymin - YPAD_BOTTOM_FRAC*yr, ymax + YPAD_TOP_FRAC*yr)

    _annotate_anchors_neat_v6(
        ax, t, ml2, idx_anchor,
        fontsize=7.0,
        dy_steps_pt=(3, -3, 4, -4, 5, -5),
        marker_safe_r_px=5.0,
        per_label_dy_pt=per_label_dy_pt,
        per_label_dy_list=per_label_dy_list,
        strict=strict,
        excluded_labels=excluded_labels,
        orient_overrides=orient_overrides,
        orient_offsets=orient_offsets
    )

    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Amplitude (mV)")

    out_dir = os.path.join(save_dir, label)
    os.makedirs(out_dir, exist_ok=True)
    out_png = os.path.join(out_dir, f"{base}_raw_points_numbers_only.png")
    out_pdf = os.path.join(out_dir, f"{base}_raw_points_numbers_only.pdf")
    out_svg = os.path.join(out_dir, f"{base}_raw_points_numbers_only.svg")

    fig.tight_layout(pad=0.4)
    fig.savefig(out_png, bbox_inches="tight")
    fig.savefig(out_pdf, bbox_inches="tight")
    fig.savefig(out_svg, bbox_inches="tight", transparent=False)
    if show:
        plt.show()
    plt.close(fig)
    return out_png

# ---------- Batch encode ----------
def encode_sampler_var_all_centered(
    root: str,
    out_dir: str = "spikes_sampler_centered_var",
    left_win=(-0.30, -0.05), right_win=(0.05, 0.45),
    n_left: int = 5, n_qrs: int = 5, n_right: int = 5,
    qrs_offsets_s: Optional[List[float]] = None,
    qrs_span_s: float = 0.020,
    left_gamma: float = 1.0, right_gamma: float = 1.0,
    slots: int = 128,
    min_spikes_per_anchor: int = 0,
    max_spikes_per_anchor: Optional[int] = None,
    sign_mode: str = "abs",
    rate_gamma: float = 1.0,
    rate_scale: float = 1.0
) -> List[str]:
    written: List[str] = []
    files = list_all_npz(root)
    if not files:
        print(f"[WARN] No input NPZ under: {root}")
        return written
    for p in files:
        x, t_rel, fs = _load_raw_numeric(p)
        label = _safe_label_from_path(p)
        S, names, meta = build_sampler_var_ml2(
            x, t_rel, fs,
            left_win=left_win, right_win=right_win,
            n_left=n_left, n_qrs=n_qrs, n_right=n_right,
            qrs_offsets_s=qrs_offsets_s, qrs_span_s=qrs_span_s,
            left_gamma=left_gamma, right_gamma=right_gamma,
            slots=slots,
            min_spikes_per_anchor=min_spikes_per_anchor,
            max_spikes_per_anchor=max_spikes_per_anchor,
            sign_mode=sign_mode,
            rate_gamma=rate_gamma,
            rate_scale=rate_scale
        )
        base = os.path.splitext(os.path.basename(p))[0]
        out_path = os.path.join(out_dir, label, f"{base}_sampler_centered_var.npz")
        save_spike_grid_npz(
            out_path, t_rel, fs, label, S, names,
            encoder=np.array("sampler_centered_var"),
            anchor_indices=meta["anchor_indices"],
            anchor_times=meta["anchor_times"],
            anchor_regions=meta["anchor_regions"],
            slots=np.int32(meta["slots"]),
            params=np.array(str({
                "left_win": left_win, "right_win": right_win,
                "n_left": n_left, "n_qrs": n_qrs, "n_right": n_right,
                "qrs_offsets_s": qrs_offsets_s, "qrs_span_s": qrs_span_s,
                "left_gamma": left_gamma, "right_gamma": right_gamma,
                "sign_mode": sign_mode,
                "min_spikes": min_spikes_per_anchor,
                "max_spikes": max_spikes_per_anchor,
                "rate_gamma": rate_gamma,
                "rate_scale": rate_scale,
                "slots": slots
            }), dtype=np.unicode_)
        )
        written.append(out_path)
    print(f"[DONE] Encoded {len(written)} files (Sampler-N×{slots} centered) → {out_dir}")
    return written

# ---------- Demo: 1 contoh per kelas ----------
def demo_plot_points_numbers_only_per_class(
    root: str, out_dir_s: str,
    seed: int = 7,
    save_dir: str = "figs_sampler_centered_var",
    left_win=(-0.30, -0.05), right_win=(0.05, 0.45),
    n_left: int = 5, n_qrs: int = 5, n_right: int = 5,
    qrs_offsets_s: Optional[List[float]] = None,
    qrs_span_s: float = 0.020,
    left_gamma: float = 1.0, right_gamma: float = 1.0,
    per_label_dy_pt: Optional[Dict[int,float]] = None,
    per_label_dy_list: Optional[Sequence[Optional[float]]] = None,
    strict: bool = False,
    excluded_labels: Sequence[int] = EXCLUDED_LABELS,
    orient_overrides: Dict[int, str] = ORIENT_OVERRIDES
):
    rng = np.random.default_rng(seed)
    classes = ["N","S","V","F","Q"]
    for c in classes:
        files = sorted(glob(os.path.join(root, c, "*.npz")))
        if not files:
            print(f"[WARN] no files for class {c}")
            continue
        pick = rng.choice(files, size=1, replace=False)[0]
        base = os.path.splitext(os.path.basename(pick))[0]
        sfile = os.path.join(out_dir_s, c, f"{base}_sampler_centered_var.npz")
        out_png = plot_raw_points_and_numbers_only(
            pick, sfile, save_dir=save_dir, show=False,
            left_win=left_win, right_win=right_win,
            n_left=n_left, n_qrs=n_qrs, n_right=n_right,
            qrs_offsets_s=qrs_offsets_s, qrs_span_s=qrs_span_s,
            left_gamma=left_gamma, right_gamma=right_gamma,
            per_label_dy_pt=per_label_dy_pt,
            per_label_dy_list=per_label_dy_list,
            strict=strict,
            excluded_labels=excluded_labels,
            orient_overrides=orient_overrides
        )
        print(f"[FIG] {c}: {out_png}")

# ---------- Run ----------
if __name__ == "__main__":
    _ = encode_sampler_var_all_centered(
        root=ROOT_DIR, out_dir=OUT_DIR,
        left_win=LEFT_WIN, right_win=RIGHT_WIN,
        n_left=N_LEFT, n_qrs=N_QRS, n_right=N_RIGHT,
        qrs_offsets_s=QRS_OFFSETS_S, qrs_span_s=QRS_SPAN_S,
        left_gamma=LEFT_GAMMA, right_gamma=RIGHT_GAMMA,
        slots=SLOTS,
        min_spikes_per_anchor=MIN_SPIKES_PER_ANCHOR,
        max_spikes_per_anchor=MAX_SPIKES_PER_ANCHOR,
        sign_mode=SIGN_MODE,
        rate_gamma=RATE_GAMMA,
        rate_scale=RATE_SCALE
    )

    demo_plot_points_numbers_only_per_class(
        ROOT_DIR, OUT_DIR, seed=SEED, save_dir=FIGS_DIR,
        left_win=LEFT_WIN, right_win=RIGHT_WIN,
        n_left=N_LEFT, n_qrs=N_QRS, n_right=N_RIGHT
    )
    print("Plotting titik & nomor (untuk data sintetis) — selesai.")


[DONE] Encoded 1500 files (Sampler-N×64 centered) → spikes_sampler_centered_var
[FIG] N: figs_sampler_var/N/beat_0283_raw_points_numbers_only.png
[FIG] S: figs_sampler_var/S/beat_0187_raw_points_numbers_only.png
[FIG] V: figs_sampler_var/V/beat_0205_raw_points_numbers_only.png
[FIG] F: figs_sampler_var/F/beat_0269_raw_points_numbers_only.png
[FIG] Q: figs_sampler_var/Q/beat_0173_raw_points_numbers_only.png
Plotting titik & nomor (untuk data sintetis) — selesai.
